In [2]:
'''!git clone https://github.com/AliaksandrSiarohin/video-preprocessing
!wget https://yt-dl.org/downloads/latest/youtube-dl -O youtube-dl
!chmod a+rx youtube-dl'''

In [1]:
import os
import pandas as pd
import subprocess
from itertools import cycle
from multiprocessing import Pool
from tqdm.auto import tqdm
DEVNULL = open(os.devnull, 'wb')
def run(paras):
    download(*paras)
def download(video_id,youtube,video_folder):
    video_id=video_id.split('#')[0]
    video_path = os.path.join(video_folder, video_id + ".mp4")
    if os.path.exists(video_path):
        return
    subprocess.call([youtube, '-f', "''best/mp4''", '--write-auto-sub', '--write-sub',
                     '--sub-lang', 'en', '--skip-unavailable-fragments',
                     "https://www.youtube.com/watch?v=" + video_id, "--output",
                     video_path], stdout=DEVNULL, stderr=DEVNULL)
    return video_path

def load_video(metadata,video_folder,youtube,workers:int=1):
    if not os.path.exists(video_folder):
        os.makedirs(video_folder)
    df = pd.read_csv(metadata)
    pool = Pool(processes=8)
    youtubes=cycle([youtube])
    video_folders=cycle([video_folder])
    video_ids = set(df['video_id'])
    for chunks_data in tqdm(pool.imap_unordered(run, zip(video_ids,youtubes,video_folders))):
        None
    

In [ ]:
load_video('./data/taichi-metadata.csv',
           'video_folder',
           './data/youtube-dl',8)

0it [00:00, ?it/s]

In [ ]:
import numpy as np
import pandas as pd
import imageio
import os
import subprocess
from multiprocessing import Pool
from itertools import cycle
import warnings
import glob
import time
from tqdm import tqdm
from argparse import ArgumentParser

warnings.filterwarnings("ignore")

DEVNULL = open(os.devnull, 'wb')

def download(video_id, args):
    video_path = os.path.join(args.video_folder, video_id + ".mp4")
    subprocess.call([args.youtube, '-f', "''best/mp4''", '--write-auto-sub', '--write-sub',
                     '--sub-lang', 'en', '--skip-unavailable-fragments',
                     "https://www.youtube.com/watch?v=" + video_id, "--output",
                     video_path], stdout=DEVNULL, stderr=DEVNULL)
    return video_path


def run(data):
    video_id, args = data
    if not os.path.exists(os.path.join(args.video_folder, video_id.split('#')[0] + '.mp4')):
       download(video_id.split('#')[0], args)

    if not os.path.exists(os.path.join(args.video_folder, video_id.split('#')[0] + '.mp4')):
       print ('Can not load video %s, broken link' % video_id.split('#')[0])
    return 
    reader = imageio.get_reader(os.path.join(args.video_folder, video_id.split('#')[0] + '.mp4'))
    fps = reader.get_meta_data()['fps']

    df = pd.read_csv(args.metadata)
    df = df[df['video_id'] == video_id]
    
    all_chunks_dict = [{'start': df['start'].iloc[j], 'end': df['end'].iloc[j],
                        'bbox': list(map(int, df['bbox'].iloc[j].split('-'))), 'frames':[]} for j in range(df.shape[0])]
    ref_fps = df['fps'].iloc[0]
    ref_height = df['height'].iloc[0]
    ref_width = df['width'].iloc[0]
    partition = df['partition'].iloc[0]
    try:
        for i, frame in enumerate(reader):
            for entry in all_chunks_dict:
                if (i * ref_fps >= entry['start'] * fps) and (i * ref_fps < entry['end'] * fps):
                    left, top, right, bot = entry['bbox']
                    left = int(left / (ref_width / frame.shape[1]))
                    top = int(top / (ref_height / frame.shape[0]))
                    right = int(right / (ref_width / frame.shape[1]))
                    bot = int(bot / (ref_height / frame.shape[0]))
                    crop = frame[top:bot, left:right]
                    if args.image_shape is not None:
                       crop = img_as_ubyte(resize(crop, args.image_shape, anti_aliasing=True))
                    entry['frames'].append(crop)
    except imageio.core.format.CannotReadFrameError:
        None

    for entry in all_chunks_dict:
        if 'person_id' in df:
            first_part = df['person_id'].iloc[0] + "#"
        else:
            first_part = ""
        first_part = first_part + '#'.join(video_id.split('#')[::-1])
        path = first_part + '#' + str(entry['start']).zfill(6) + '#' + str(entry['end']).zfill(6) + '.mp4'
        save(os.path.join(args.out_folder, partition, path), entry['frames'], args.format)


video_folder='video_folder'
metadata='./video-preprocessing/taichi-metadata.csv'
if not os.path.exists(video_folder):
    os.makedirs(video_folder)
df = pd.read_csv(metadata)
video_ids = set(df['video_id'])
pool = Pool(processes=8)
args_list = cycle([args])
for chunks_data in tqdm(pool.imap_unordered(run, zip(video_ids, args_list))):
    None  
